# Inversion cheat sheet

This is a summary cheet sheet which combines the key principles into one easily accessible format for the purpose of revision and access during the class test.

# Table of contents
#### [Lecture 1: ](#lecture1)
#### [Lecture 2: ](#lecture2)
#### [Lecture 3: ](#lecture3)
#### [Lecture 4: ](#lecture4)
<hr/>

## Lecture 1: forward vs inver problems, linear algebra <a name="lecture1"></a>

In STEM, we must pose many problems as inversion or optimsations problems. In the most general sense, inversion asks for some function 

$$y=f(x),$$

given $y$, what is $x$? This is contrary to the forward problem, which asks: given $x$, what is $y$? Thus, inversion problems can be formulated as 

$$f(x) = y,$$ 

or more commonly as 

$$A\mathbf{x}=\mathbf{b},$$ 

for a system of equations. Often, we choose to simplify our problem by approximating non-linear terms with linear functions. Then, it is important to remember that a linear function commutes with addition, i.e., 

$$f(\mathbf{x+y}) = f(\mathbf{x})+f(\mathbf{y}),$$
$$f(\alpha \mathbf{x})=\alpha f(\mathbf{x}), \ \alpha \in \mathbb{R}.$$

### Newton's method

We can often pose an inversion problem as a root finding problem by solving iteratively for $x_{i+1}$. If we have some function $f$, then we can find a root by setting $f(x)=0$ and solving for $x$. Then, in a similar way, if we have $g(x)=f'(x)$ then the $x$ for which $g(x)=0$ are minimum solutions to the problem. By posing the problem in this manner, we can apply Newton's method to   

$$x_{i+1}=x_i - \frac{f(x_i)/f'(x_i)}$$

## Lecture 2: <a name="lecture2"></a>

## Lecture 3: <a name="lecture3"></a>

We can often pose an inversion problem as a root finding problem by solving iteratively for $x_{i+1}$. If we have some function $f$, then we can find a root by setting $f(x)=0$ and solving for $x$. Then, in a similar way, if we have $g(x)=f'(x)$ then the $x$ for which $g(x)=0$ are minimum solutions to the problem. By posing the problem in this manner, we can apply Newton's method to

## Lecture 4: <a name="lecture4"></a>